### package & fetch datas

In [31]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from scipy import stats

df = pd.read_csv("fake_data.csv")
df = df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df['RSI'] = ta.rsi(df.close, length=14)
df['EMA'] = ta.ema(df.close, length=150)
df.tail()
df=df[:5000]

### Set Trend

In [32]:
ema_signal = [0]*len(df)
history_candles = 15

for row in range(history_candles, len(df)):
    up = 1
    down = 1
    for i in range(row-history_candles, row+1):
        if max(df.open[i], df.close[i]) >= df.EMA[i]:
            down = 0
        if min(df.open[i], df.close[i]) <= df.EMA[i]:
            up = 0
    if up == 1 & down == 1:
        ema_signal[row] = 3
    elif up == 1:
        ema_signal[row] == 2   
    elif down == 1:
        ema_signal[row] = 1
                 
df["ema_signal"] = ema_signal                

In [33]:
def isPivot(candle, window):
    if candle - window < 0 or candle + window >= len(df):
        return 0
    high_pivot = 1
    low_pivot = 2
    for i in range(candle - window, candle + window + 1):
        if df.iloc[candle].low > df.iloc[i].low:
            low_pivot = 0
        if df.iloc[candle].high < df.iloc[i].high:
            high_pivot = 0
    if (high_pivot & low_pivot):
        return 3
    elif high_pivot:
        return high_pivot
    elif low_pivot:
        return low_pivot
    else:
        return 0
                

In [34]:
window = 15
df['isPivot'] = df.apply(lambda x: isPivot(x.name, window), axis = 1)

In [35]:
def point_position(x):
    if x['isPivot'] == 2:
        return x['low'] - 1e-3
    elif x['isPivot'] == 1:
        return x['high'] + 1e-3
    else:
        return np.nan
df['point_position'] = df.apply(lambda row: point_position(row), axis = 1)    

In [36]:
dfpl = df[600:1000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['point_position'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [37]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    pattern_detected = False

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2] < idxhighs[2])
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        pattern_1 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] > lows[1] and lows[2] < highs[1] and
            highs[2] < highs[1] and highs[2] > lows[2]
            )

        pattern_2 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] < highs[1] 
            )

        if (order_condition and
            diff_condition and
            (pattern_1 or pattern_2)
        ):
            pattern_detected = True

    if pattern_detected:
        return 1
    else:
        return 0

In [46]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=16))


In [47]:
df[df['pattern_detected']!=0]


,Gmt time,open,high,low,close,volume,RSI,EMA,ema_signal,isPivot,point_position,pattern_detected
